# PyTorch_Tutorial_2 : Autograd: Automatic Differentiation

### 이 노트는 아래의 웹사이트의 코드를 따라가며 해설하였다.

https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html

- Central to all neural networks in PyTorch is the autograd package
- The autograd package provides automatic differentiation for all operations on Tensors.

In [1]:
import torch

### 1. Tracking all operations on tensors

- 텐서의 requires_grad 속성을 True로 설정하면, 대상 텐서에 가해지는 모든 연산을 추적한다.

In [2]:
# If you set its attribute .requires_grad as True,
# it starts to track all operations on it. 
x = torch.ones(2, 2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [3]:
# grad_fn 속성은 대상 텐서를 만들어낸 연산에 대한 정보를 담고 있다.
# 여기서, y는 x에 2를 더하는 연산을 통해 만들어냈음을 알 수 있다.
y = x + 2
print(y)
print(y.grad_fn)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [4]:
z = y * y * 3     # 곱하기를 통해 z 텐서를 만들었다.
out = z.mean()    # 평균계산을 통해 out 텐서를 만들어냈다.
print(z, '\n', out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) 
 tensor(27., grad_fn=<MeanBackward0>)


In [5]:
# PyTorch에서 "_"로 끝나는 메소드는 대상을 변화시킨다(In-place).
# 따라서, a.requires_grad_()는 텐서 a의 requires_grad 속성값을 바꾼다.
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print("a =", a)
print(a.requires_grad, '\n')
a.requires_grad_(True)
print("a =", a)
print(a.requires_grad, '\n')
b = (a * a).sum()
print(b.grad_fn)

a = tensor([[-0.7979,  0.9563],
        [ 0.1155,  0.5435]])
False 

a = tensor([[-0.7979,  0.9563],
        [ 0.1155,  0.5435]], requires_grad=True)
True 



### 2. Gradients

#### NOTE ) 야코비행렬(Jacobian Marix)이나 vector-Jacobian product에 대한 자세한 설명은 위 사이트 참조

- backward( )메소드를 통해 대상 텐서와 관련된 모든 gradients가 자동으로 계산된다.

In [6]:
# out = 3*(x+2)**2 를 위에서 계산하였다.
# out은 scalar이므로, backward()메소드 사용시 argument에 아무값도 넣지 않아도 된다(디폴트로 torch.tensor(1)이 적용됨).
out.backward()

In [7]:
# 다음의 결과를 이해할 것!
# 텐서 x의 모든 요소들을 변수로 두고 out을 각각의 변수에 대해 편미분한 후,
# 텐서 x의 요소값들을 대입해 편미분값을 구한 것
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


#### backward( ) 메소드의 대상텐서가 스칼라가 아니라면?

- PyTorch의 autograd 패티지는 사실 vector-Jacobian 곱을 구한다.
- 이는 야코비행렬(Jacobian Matrix)에 사용자가 backward의 argument로 대입한 vector의 product를 말한다.
- 대상 텐서가 스칼라일 경우에 야코비행렬은 스칼라의 gradient가 되고, backward의 argument를 비워놓으면 자동으로 torch.tensor(1)이 대입되므로 최종적으로 vector-Jacobian 곱이 스칼라의 gradient가 된다. 따라서 위에서는 간단히 "backward가 gradient를 구해준다"고 한 것이다.
- 스칼라가 아닌 경우는 아래의 예시를 참고해보자.

In [8]:
x = torch.randn(3, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
    y = y * 2

print("NORM : ", y.data.norm())
print(y)

NORM :  tensor(1844.2112)
tensor([  434.0550,   120.0645, -1788.3779], grad_fn=<MulBackward0>)


In [9]:
# Now in this case y is no longer a scalar.
# torch.autograd could not compute the full Jacobian directly,
# but if we just want the vector-Jacobian product,
# simply pass the vector to backward as argument:
v = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(v)

print(x.grad)

tensor([4.0960e+02, 4.0960e+03, 4.0960e-01])


In [10]:
# 연산추적을 멈추고 싶을 때
print(x.requires_grad)
print((x ** 2).requires_grad)

with torch.no_grad():
    print((x ** 2).requires_grad)

True
True
False


In [11]:
# detach()를 이용할 수도 있다.
print(x.requires_grad)
y = x.detach()
print(y.requires_grad)
print(x.eq(y).all())    # 성분은 모두 같다.

True
False
tensor(True)
